In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import os
from catboost import CatBoostClassifier, Pool

# Настройки отображения
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', 30)
pd.options.display.float_format = '{:.2f}'.format

# Инициализация путей
root = Path(os.getcwd()).parent.parent
data_in = root / 'data' / 'processed_data'
data_out = root / 'data' / 'model_data'

print("Пути инициализированы:")
print(f"Processed data: {data_in}")
print(f"Model data: {data_out}")

Пути инициализированы:
Processed data: /Users/aleksey.sushchikh/Desktop/GitHub/MIFIHackatonSberAutoSubscriptionAnalysis/data/processed_data
Model data: /Users/aleksey.sushchikh/Desktop/GitHub/MIFIHackatonSberAutoSubscriptionAnalysis/data/model_data


In [3]:
# Загрузка данных
print("\nЗагружаем предобработанные данные...")
data = pd.read_pickle(data_in / 'test.pkl')
print("\nДанные успешно загружены!")


Загружаем предобработанные данные...

Данные успешно загружены!


In [6]:
 def generate_summary(df: pd.DataFrame, name: str, sample_size: int = 3) -> None:
    """Генерация расширенной сводки по данным примерами"""
    print(f"\n{'='*50} {name.upper()} {'='*50}")
    print(f"Общее количество записей: {df.shape[0]:,}")
    print(f"Количество признаков: {df.shape[1]}")
    
    # Типы данных
    print("\nТипы данных:")
    print(df.dtypes.value_counts().rename('count').to_frame())
    
    # Пропуски
    missing = df.isna().sum().sort_values(ascending=False)
    missing_pct = (missing / df.shape[0] * 100).round(2)
    missing_df = pd.concat([missing, missing_pct], axis=1, keys=['count', '%']).query('count > 0')
    if not missing_df.empty:
        print("\nПропущенные значения:")
        print(missing_df)
    else:
        print("\nПропущенных значений нет")
        
    # Дубликаты
    dupes = df.duplicated().sum()
    print(f"\nДубликаты: {dupes} ({dupes/df.shape[0]*100:.2f}%)")
    
    # Примеры
    print(f"\nПервые {sample_size} записей:")
    display(df.head(sample_size))


def analyze_column(df: pd.DataFrame, col: str, max_display: int = 50) -> None:
    """Полный анализ колонки с выводом всех уникальных значений"""
    print(f"\n{'-'*60}")
    print(f"Полный анализ колонки: {col}")
    
    # Проверка существования колонки
    if col not in df.columns:
        print(f"Колонка {col} не найдена!")
        return
    
    # Пропуски
    na_count = df[col].isna().sum()
    print(f"Пропуски: {na_count} ({na_count/len(df)*100:.1f}%)")

    # Количество уникальных значений
    unique_count = df[col].nunique(dropna=False)
    print(f"Уникальных значений: {unique_count}")
    
    # Вывод всех значений для категориальных данных
    if unique_count <= max_display:
        print("\nВсе значения:")
        print(df[col].unique())
    else:
        print(f"\nСлишком много значений (> {max_display}). Примеры:")
        print(df[col].dropna().sample(10).unique())
        
    # Частотный анализ для числовых колонок
    if pd.api.types.is_numeric_dtype(df[col]):
        print("\nОписательная статистика:")
        print(df[col].describe())
    else:
        print("\nТоп-10 значений:")
        print(df[col].value_counts(dropna=False).head(10))

In [8]:
# Объявляем целевые действия
target_actions = [
    # Отправка форм и заявок
    'sub_submit_success', 'sub_submit_error',          # sub_submit
    'form_request_call_sent',                          # phone
    'sub_callback_submit_click',                       # sub_button_click
    'sub_car_request_submit_click',                    # sub_button_click
    'sub_custom_question_submit_click',                # sub_button_click
    'sub_car_claim_submit_click',                      # sub_button_click

    # Обратный звонок и контакт
    'click_on_request_call', 'click_on_phone',          # phone
    'phone_entered_on_form_request_call',               # phone
    'showed_form_request_call',                         # phone

    # Чат и онлайн-диалог
    'start_chat', 'chat requested',                     # chat, jivosite
    'user_message', 'client initiate chat',             # chat, jivosite
    'callback requested',                               # jivosite
    'offline form shown',                               # jivosite

    # Аутентификация (сильный признак санкционированного взаимодействия)
    'phone_auth_success', 'sber_id_auth_success',       # auth

    # Кредит и рассчет
    'calculate',                                        # credit_landing_pos
    'request', 'request_success',                       # credit_landing_pos

    # Клики по покупке/оформлению подписки
    'click_on_subscription',                            # header, main_banners
    'click_auto_subscription',                          # main_services
    'click_free_car_selection',                         # main_services, chat
    'click_buy_auto',                                   # main_advantages, main_services
    'click_auto',                                       # main_services
    'click_car_selection',                              # main_services
    'click_car_buyback',                                # header, main_services
    'click_setelem_credit',                             # main_services
    'click_pos_credit',                                 # main_services

    # Публикация объявления (для частных продавцов)
    'success_ad_creation',                              # add_ad_publish_success

    # Запросы по VIN, цене, городу — сильный признак готовности
    'forward_to_price', 'sap_search_form_cost_to',      # add_ad_city, sap_search_form
    'sap_search_form_cost_from',                       # sap_search__form

    # Переходы из листинга и карточки
    'go_to_car_card',                                   # listing_ads, greenday_listing_ads
    'sub_view_cars_click',                              # sub_button_click, greenday_gtm.triggergroup

    # Клики из “подписаться”
    'sub_open_dialog_click',                            # sub_button_click, greenday_sub_button_click

    # Дополнительные целевые события
    'request',                                          # credit_landing_pos (повтор)
    'order_call',                                       # если встречается в схеме
]

data['is_target'] = data['event_action'].isin(target_actions).astype('int')

generate_summary(data, "Данные")
for col in data.columns:
    analyze_column(data, col, max_display=100)


================================================== ДАННЫЕ ==================================================
Общее количество записей: 15,854,246
Количество признаков: 29

Типы данных:
         count
object      25
float64      3
int64        1

Пропущенные значения:
                             count      %
event_value               15854246 100.00
device_model              15730683  99.22
utm_keyword                9307387  58.71
hit_time                   9288098  58.58
device_os                  9255807  58.38
hit_referer                6402580  40.38
event_label                3887960  24.52
utm_adcontent              2905147  18.32
utm_campaign               2264440  14.28
device_brand               1406759   8.87
event_action                127776   0.81
event_category              127776   0.81
hit_page_path               127776   0.81
hit_type                    127776   0.81
hit_number                  127776   0.81
hit_date                    127776   0.81
utm_source       

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city,hit_date,hit_time,hit_number,hit_type,hit_referer,hit_page_path,event_category,event_action,event_label,event_value,is_target
0,1000009318903347362.163266...,232832813.1632663714,2021-09-26,16:00:00,1.00,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,PkybGvWbaqORmxjNunqZ,NaN,mobile,None,Samsung,NaN,412x869,Chrome,Russia,Gelendzhik,2021-09-26,NaN,15.00,event,HbolMJUevblAbkHClEQa,podpiska.sberauto.com/faq?...,sub_page_view,sub_faq,KclpemfoHstknWHFiLit,None,0
1,1000009318903347362.163266...,232832813.1632663714,2021-09-26,16:00:00,1.00,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,PkybGvWbaqORmxjNunqZ,NaN,mobile,None,Samsung,NaN,412x869,Chrome,Russia,Gelendzhik,2021-09-26,NaN,3.00,event,HbolMJUevblAbkHClEQa,podpiska.sberauto.com/,sub_page_view,sub_landing,KclpemfoHstknWHFiLit,None,0
2,1000009318903347362.163266...,232832813.1632663714,2021-09-26,16:00:00,1.00,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,PkybGvWbaqORmxjNunqZ,NaN,mobile,None,Samsung,NaN,412x869,Chrome,Russia,Gelendzhik,2021-09-26,NaN,4.00,event,HbolMJUevblAbkHClEQa,podpiska.sberauto.com/,listing_ads,go_to_car_card,hAHqGICPFQiPwtzubOzs,None,1



------------------------------------------------------------
Полный анализ колонки: session_id
Пропуски: 0 (0.0%)
Уникальных значений: 1862386

Слишком много значений (> 100). Примеры:
['7364072604327003426.1629013286.1629013286'
 '6463262982762317232.1635435954.1635435954'
 '1522768149475896693.1625846928.1625846928'
 '5314216535080831161.1627900419.1627900472'
 '1017030948615564254.1640967135.1640967135'
 '675837157473929879.1631472279.1631472279'
 '5836906139387411878.1635499430.1635499430'
 '2069057727194157792.1630825804.1630825804'
 '536440961643253619.1639383543.1639383543'
 '4563409153123369468.1622098427.1622098772']

Топ-10 значений:
session_id
5442565791571325612.1632449195.1632449195    768
6568868914238486437.1632270313.1632270313    678
5959671972744778783.1632490527.1632490600    548
7452598043578978502.1632358598.1632358598    514
3070792010704358528.1629752408.1629752408    498
686125592720823356.1634587719.1634587719     496
8115026869866033734.1629319807.1629319807 

In [9]:
# X и y
X = data.drop('is_target', axis=1)
y = data['is_target']

# создаём Pool (рекомендуется при наличии категориальных)
cat_feats = ['session_id','client_id','utm_source','utm_medium',
             'utm_campaign', 'utm_adcontent', 'utm_keyword',
             'device_category', 'device_os', 'device_brand', 'device_browser',
             'geo_country',	'geo_city', 'hit_referer',
             'hit_page_path', 'event_category', 'event_action', 'event_label']
train_pool = Pool(X, y, cat_features=cat_feats)

# обучение
model = CatBoostClassifier(
    iterations=100,
    learning_rate=0.1,
    depth=6,
    verbose=50
)
model.fit(train_pool)

CatBoostError: Invalid type for cat_feature[non-default value idx=24973,feature_idx=1]=nan : cat_features must be integer or string, real number values and NaN values should be converted to string.